In [1]:
import pandas as pd
import numpy as np

In [2]:
!ls

1. Data Cleaning (Basic).ipynb
2. Data Analysis and Cleaning (Advance).ipynb
3. Data Prep.ipynb
4. Model_Training.ipynb
data_analysis_ride_request.html


#### Reading DataSet

In [3]:
df = pd.read_csv('../Data/raw_data.csv', low_memory = False, compression='gzip')
len(df)

8381556

In [4]:
df.head()

,ts,number,pick_lat,pick_lng,drop_lat,drop_lng
0,2020-03-26 07:07:17,14626,12.313621,76.658195,12.287301,76.602280
1,2020-03-26 07:32:27,85490,12.943947,77.560745,12.954014,77.543770
2,2020-03-26 07:36:44,05408,12.899603,77.587300,12.934780,77.569950
3,2020-03-26 07:38:00,58940,12.918229,77.607544,12.968971,77.636375
4,2020-03-26 07:39:29,05408,12.899490,77.587270,12.934780,77.569950


### A Customer_ID `number` at a particular timestamp can only have one entry
### Removing Duplicate Entries ['ts','number']

In [5]:
df[df.duplicated(subset=['ts','number'],keep=False)]

,ts,number,pick_lat,pick_lng,drop_lat,drop_lng
235,2020-03-26 18:10:35,16795,12.967236,77.641594,13.014504,77.650856
236,2020-03-26 18:10:35,16795,12.967236,77.641594,13.014504,77.650856
407,2020-03-26 21:35:50,65856,12.917173,77.586400,12.913940,77.685280
408,2020-03-26 21:35:50,65856,12.917173,77.586400,12.913940,77.685280
443,2020-03-26 23:26:29,27554,12.933715,77.619300,12.938208,77.587520
...,...,...,...,...,...,...
8381231,2021-03-26 22:23:12,61636,12.975229,77.620370,13.017285,77.618200
8381245,2021-03-26 22:25:13,61636,12.975229,77.620370,13.017285,77.618200
8381246,2021-03-26 22:25:13,61636,12.975229,77.620370,13.017285,77.618200
8381248,2021-03-26 22:25:27,61636,12.975229,77.620370,13.017285,77.618200


### There are 113540 Duplicate Entries
#### We have 8315498 Unique timestamp, customer_id rows. 

In [6]:
## Keeping first occurence
df.drop_duplicates(subset=['ts','number'], inplace = True, keep = 'last')

df.reset_index(inplace = True, drop = True)

In [7]:
# Info of Dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8315498 entries, 0 to 8315497
Data columns (total 6 columns):
ts          object
number      object
pick_lat    float64
pick_lng    float64
drop_lat    float64
drop_lng    float64
dtypes: float64(4), object(2)
memory usage: 380.7+ MB


In [8]:
#Count missing values
np.count_nonzero(df.isnull().values)

0

In [9]:
df['number'] = pd.to_numeric(df['number'], errors = 'coerce')

#Count missing values
np.count_nonzero(df.isnull().values)

116

#### There are 116 NaN rows, dropping NaN rows.

In [10]:
df.dropna(inplace = True)
len(df)

8315382

In [11]:
df['number'] = pd.to_numeric(df['number'], errors = 'coerce', downcast='integer')
df['ts'] = pd.to_datetime(df['ts'])

In [12]:
# Info of Dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8315382 entries, 0 to 8315497
Data columns (total 6 columns):
ts          datetime64[ns]
number      int32
pick_lat    float64
pick_lng    float64
drop_lat    float64
drop_lng    float64
dtypes: datetime64[ns](1), float64(4), int32(1)
memory usage: 412.4 MB


### Breaking Time to Features

In [13]:
df['hour'] = df['ts'].dt.hour
df['mins'] = df['ts'].dt.minute
df['day'] = df['ts'].dt.day
df['month'] = df['ts'].dt.month
df['year'] = df['ts'].dt.year
df['dayofweek'] = df['ts'].dt.dayofweek

In [14]:
df.to_csv('../Data/data_checkpoint/preprocessed_1.csv',index = False, compression = 'gzip')

In [15]:
df

,ts,number,pick_lat,pick_lng,drop_lat,drop_lng,hour,mins,day,month,year,dayofweek
0,2020-03-26 07:07:17,14626,12.313621,76.658195,12.287301,76.602280,7,7,26,3,2020,3
1,2020-03-26 07:32:27,85490,12.943947,77.560745,12.954014,77.543770,7,32,26,3,2020,3
2,2020-03-26 07:36:44,5408,12.899603,77.587300,12.934780,77.569950,7,36,26,3,2020,3
3,2020-03-26 07:38:00,58940,12.918229,77.607544,12.968971,77.636375,7,38,26,3,2020,3
4,2020-03-26 07:39:29,5408,12.899490,77.587270,12.934780,77.569950,7,39,26,3,2020,3
...,...,...,...,...,...,...,...,...,...,...,...,...
8315493,2021-03-26 23:55:24,50410,12.907856,77.557870,12.954270,77.530785,23,55,26,3,2021,4
8315494,2021-03-26 23:58:15,12580,12.981010,77.694450,12.969070,77.704280,23,58,26,3,2021,4
8315495,2021-03-26 22:11:20,72339,12.924252,77.650520,12.905820,77.630570,22,11,26,3,2021,4
8315496,2021-03-26 22:12:30,72339,12.924252,77.650520,12.905820,77.630570,22,12,26,3,2021,4
